# <center>Optimisation - II</center>
### <center>Stochastic Search, GA and PSO</center>

In [99]:
%matplotlib 
import numpy as np
import matplotlib.pyplot as plt

Using matplotlib backend: MacOSX


## Stochastic Search

#### Objective function

$$ maximize \space\space
Z(x,y) = 1.7*exp\bigg[-\bigg\{\dfrac{{(x-3)}^2}{10}+\dfrac{{(y-3)}^2}{10}\bigg\}\bigg]+
exp\bigg[-\bigg\{\dfrac{{(x+5)}^2}{8}+\dfrac{{(y+5)}^2}{8}\bigg\}\bigg] +\\ 2*exp\bigg[-\bigg\{\dfrac{{x}^2}{4}+\dfrac{{y}^2}{5}\bigg\}\bigg] +
1.5*exp\bigg[-\bigg\{\dfrac{{(x-4)}^2}{18}+\dfrac{{(y+4)}^2}{16}\bigg\}\bigg]+\\
1.2*exp\bigg[-\bigg\{\dfrac{{(x+4)}^2}{18}+\dfrac{{(y-4)}^2}{16}\bigg\}\bigg]
$$

In [100]:
def objective_function(x,y):
    z=1.7*np.exp(-(((x-3)**2)/10+((y-3)**2)/10))+np.exp(-(((x+5)**2)/8+((y+5)**2)/8))+2*np.exp(-(((x)**2)/4+((y)**2)/5))+1.5*np.exp(-(((x-4)**2)/18+(y+4)**2/16))+1.2*np.exp(-(((x+4)**2)/18+((y-4)**2)/16))
    return z

In [101]:
def initialise(Xmin,Xmax,popsize):
    x = np.random.uniform(low=Xmin[0],high=Xmax[0],size=(popsize,1))
    y = np.random.uniform(low=Xmin[1],high=Xmax[1],size=(popsize,1))
    return np.hstack((x,y))

In [102]:
def NRRI(Xmin,Xmax,nrri):
    x = np.random.uniform(low=Xmin[0],high=Xmax[0],size=(nrri,1))
    y = np.random.uniform(low=Xmin[1],high=Xmax[1],size=(nrri,1))
    return np.hstack((x,y))

In [103]:
def NRLC(data, nrlc, k=10):    ## Selecting parents using tournment selection method
    data_nrlc = np.empty((nrlc,2))
    for i in range(nrlc):
        rand_idx1 = np.random.choice(len(data),k,replace=False)
        rand_idx2 = np.random.choice(len(data),k,replace=False)
        tournament_1 = data[rand_idx1]
        tournament_2 = data[rand_idx2]
        y_tournament_1 = np.array([objective_function(x,y) for x,y in zip(tournament_1[:,0],tournament_1[:,1])])
        y_tournament_2 = np.array([objective_function(x,y) for x,y in zip(tournament_2[:,0],tournament_2[:,1])])
        p1 = np.argmax(y_tournament_1)
        p2 = np.argmax(y_tournament_2)
        eta = np.random.rand()
        data_nrlc[i,0] = data[p1,0]*eta + (1-eta)*data[p2,0]
        data_nrlc[i,1] = data[p1,1]*eta + (1-eta)*data[p2,1]
    return data_nrlc

In [104]:
def contour_plot(objective_function,Xmin,Xmax,data,itr):
    xx = np.linspace(Xmin[0],Xmax[0],100)
    yy = np.linspace(Xmin[1],Xmax[1],100)
    XX,YY = np.meshgrid(xx,yy)
    ZZ = np.empty((100,100))
    for i in range(100):
        for j in range(100):
            ZZ[i,j] = objective_function(XX[i,j],YY[i,j])

    fig, ax = plt.subplots(figsize=(8,6))
    cp = ax.contour(XX,YY,ZZ)
    fig.colorbar(cp)
    ax.scatter(data[:,0],data[:,1],s=10,color='red')
    ax.set_title('Stochastic Search - Iteration:{}'.format(itr+1))
    ax.set_xlabel('x-axis')
    ax.set_ylabel('y-axis')
    plt.show()
    plt.pause(1)
    plt.clf()

In [105]:
def stochastic_search(Xmin,Xmax,popsize=100,rho=0.5,nrri=50,nrlc=50,maxitr=10):
    data = initialise(Xmin,Xmax,popsize)

    for itr in range(maxitr):
        data_new = data.copy()
        f = np.array([objective_function(x,y) for x,y in zip(data[:,0],data[:,1])])
        u = f - min(f)
        num_child = (u*popsize/sum(u)).astype(int)
        for i in range(popsize):
            data_child = np.empty((num_child[i],2))
            for j in range(num_child[i]):
                theta = np.random.uniform(-np.pi,np.pi)
                radius = np.random.uniform(0,rho)
                data_child[j,0] = data[i,0] + radius*np.cos(theta)
                data_child[j,1] = data[i,1] + radius*np.sin(theta)
            
            data_new = np.append(data_new,data_child,axis=0)
        
        data_nrri = NRRI(Xmin,Xmax,nrri)
        data_new = np.append(data_new,data_nrri,axis=0)
        data_nrlc = NRLC(data,nrlc)
        data_new = np.append(data_new,data_nrlc,axis=0)

        f = np.array([objective_function(x,y) for x,y in zip(data_new[:,0],data_new[:,1])])
        sort_index = np.argsort(f)[::-1]   ## convert it in descending order
        data_new = data_new[sort_index]

        contour_plot(objective_function,Xmin,Xmax,data,itr)
        data = data_new[:popsize]
    plt.close('all')
    return data[0], f[0]

In [106]:
Xmin = np.array([-10,-10])
Xmax = np.array([10,10])

X_best,y_best = stochastic_search(Xmin,Xmax,popsize=100,rho=0.5,maxitr=10,nrri=30,nrlc=40)
print("Best Solution:{}".format(X_best))
print("Best Functional Value:{}".format(y_best))

Best Solution:[0.21007484 0.21686314]
Best Functional Value:2.725931888128


## Solution: Travelling Salesman Problem using Stochastic Search

In [107]:
dist_matr = np.genfromtxt('dist_matr.csv',delimiter=',')
print(dist_matr)

[[  0.  83.  93. 129. 133. 139. 151. 169. 135. 114. 110.  98.  99.  95.
   81. 152. 159. 181. 172. 185. 147. 157. 185. 220. 127. 181.]
 [ 83.   0.  40.  53.  62.  64.  91. 116.  93.  84.  95.  98.  89.  68.
   67. 127. 156. 175. 152. 165. 160. 180. 223. 268. 179. 197.]
 [ 93.  40.   0.  42.  42.  49.  59.  81.  54.  44.  58.  64.  54.  31.
   36.  86. 117. 135. 112. 125. 124. 147. 193. 241. 157. 161.]
 [129.  53.  42.   0.  11.  11.  46.  72.  65.  70.  88. 100.  89.  66.
   76. 102. 142. 156. 127. 139. 155. 180. 228. 278. 197. 190.]
 [133.  62.  42.  11.   0.   9.  35.  61.  55.  62.  82.  95.  84.  62.
   74.  93. 133. 146. 117. 128. 148. 173. 222. 272. 194. 182.]
 [139.  64.  49.  11.   9.   0.  39.  65.  63.  71.  90. 103.  92.  71.
   82. 100. 141. 153. 124. 135. 156. 181. 230. 280. 202. 190.]
 [151.  91.  59.  46.  35.  39.   0.  26.  34.  52.  71.  88.  77.  63.
   78.  66. 110. 119.  88.  98. 130. 156. 206. 257. 188. 160.]
 [169. 116.  81.  72.  61.  65.  26.   0.  37.  59.  75

In [108]:
print(dist_matr.shape)

(26, 26)


In [109]:
print((dist_matr == dist_matr.T).all())

True


### Define the Distance function

In [110]:
def dist_func(path, dist_matr):
    total_distance = 0
    i = 0
    for j in path:
        total_distance += dist_matr[j,i]
        i = j
    total_distance += dist_matr[i,0]
    return total_distance

### Function to swap two elements

In [111]:
def swap(x):
    sample_idx = np.random.choice(len(x),2,replace=False)
    x[sample_idx[0]], x[sample_idx[1]] = x[sample_idx[1]], x[sample_idx[0]]
    return x

### Initialize Population

In [112]:
def init_pop(path, popsize):
    population = []
    for i in range(popsize):
        new = np.random.permutation(path)
        population.append(new)
    return np.matrix(population)

### Random re-initialize points

In [113]:
def rrI(arr, nrri):
    rri = []
    for i in range(nrri):
        new = np.random.permutation(arr)
        rri.append(new)
    return np.matrix(rri)

### Main function for Optimum path

In [114]:
def Optimum_path(path, dist_matr, popsize=100, nrri=30, maxitr=10):
    population = init_pop(path, popsize)

    for itr in range(maxitr):
        y_arr = np.array([dist_func(np.ravel(row), dist_matr) for row in population])
        u_arr = y_arr - np.min(y_arr)
        child_num = (u_arr*popsize/np.sum(u_arr)).astype(int)
        child_arr = []
        for i in range(popsize):
            x = np.ravel(population[i,:])
            for j in range(child_num[i]):
                c = swap(x)
                child_arr.append(c)
        
        child_mat = np.matrix(child_arr)
        rri = rrI(path, nrri)

        total_pop = np.vstack((population,child_mat,rri))
        y_arr = np.array([dist_func(np.ravel(row), dist_matr) for row in total_pop])

        sorted_idx = np.argsort(y_arr)
        useful_idx = sorted_idx[:popsize]
        y_arr_sorted = y_arr[sorted_idx][:popsize]
        # print("Iteration: {}".format(itr+1))
        # print("Best Path till now is {}".format(total_pop[useful_idx[0]]))
        # print("Best Distance till now is {}".format(y_arr[useful_idx[0]]))
        poplist = []
        for i in range(len(useful_idx)):
            idx = useful_idx[i]
            popmember = np.ravel(total_pop[idx,:])
            poplist.append(popmember)
        population = np.matrix(poplist)
    return np.ravel(population[0]), y_arr_sorted[0]

In [115]:
path = np.arange(1,26)
path

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25])

In [116]:
Optimum_path(path,dist_matr,popsize=500,nrri=100,maxitr=500)

(array([24, 22, 23, 20, 25, 21, 17, 15, 14, 10,  1,  2,  3,  4,  5,  7,  6,
        12, 11, 13,  8, 19, 18,  9, 16]),
 1495.0)

In [117]:
## Optimum route for this distance matrix
ans = np.array([24,23,22,25,21,20,16,17,19,18,15,10,11,12,14,13,9,8,7,6,4,5,3,2,1])
print("The minimum distance path is:")
print(str(0)+"->",end="")
for i in range(len(ans)):
    print(str(ans[i])+"->",end="")
print(str(0))
print("Minimum possible distance is {}".format(dist_func(ans,dist_matr)))

The minimum distance path is:
0->24->23->22->25->21->20->16->17->19->18->15->10->11->12->14->13->9->8->7->6->4->5->3->2->1->0
Minimum possible distance is 937.0


In [121]:
from sklearn.manifold import MDS #Multi-dimensional scaling
model = MDS(n_components=2,dissimilarity='precomputed',random_state=1)
out = model.fit_transform(dist_matr)

n = np.arange(26)
x = out[:,0]
y = out[:,1]
fig,ax = plt.subplots(figsize=(10,8))
ax.scatter(x,y)
ax.arrow(x[0],y[0],x[ans[0]]-x[0],y[ans[0]]-y[0])
for i,txt in enumerate(n):
    if(i<24):
        ax.arrow(x[ans[i]],y[ans[i]],x[ans[i+1]]-x[ans[i]],y[ans[i+1]]-y[ans[i]])
    ax.annotate(txt,(x[i],y[i]))
plt.title("TSP Solution")
plt.show()

## Genetic Algorithm

### Define Objective Function

In [21]:
# Objective function for optimization
def obj_function(x,y):  # Ackley's Function
    z = -20* np.exp(-0.2* np.sqrt(0.5* (x**2 + y**2))) - np.exp(0.5* (np.cos(2* np.pi* x) + (np.cos(2* np.pi* y)))) + np.exp(1) + 20
    return z

### Initialise the Population

In [22]:
def initialise(Xmin,Xmax,popsize):
    x = np.random.uniform(low=Xmin[0],high=Xmax[0],size=(popsize,1))
    y = np.random.uniform(low=Xmin[1],high=Xmax[1],size=(popsize,1))
    return np.hstack((x,y))

In [23]:
def decimal_to_binary(n:int) -> str:
    return bin(n)[2:]

In [24]:
def binary_to_decimal(n:str) -> int:
    if n.startswith('b'):
        return -int(n[1:],base=2)
    else:
        return int(n, base=2)

In [57]:
def pad_zeros(bin1:str, bin2:str):
    flag1=False; flag2=False;
    if bin1.startswith('b'):
        bin1 = bin1[1:]
        flag1 = True
    if bin2.startswith('b'):
        bin2 = bin2[1:]
        flag2 = True
    m = len(bin1)
    n = len(bin2)
    if m != n:
        if m > n:
            bin2 = bin2.rjust(m, '0')
        else:
            bin1 = bin1.rjust(n, '0')
    if flag1 == True:
        bin1 = 'b'+bin1
    if flag2 == True:
        bin2 = 'b'+bin2
    return bin1, bin2

### Crossover between two parents

In [58]:
def crossover(bin1:str, bin2:str):
    flag1=False; flag2=False;
    if bin1.startswith('b'):
        bin1 = bin1[1:]
        flag1=True
    if bin2.startswith('b'):
        bin2 = bin2[1:]
        flag2=True
    rand_idx = np.random.choice(len(bin1))
    mod_bin1 = bin1[:rand_idx] + bin2[rand_idx:]
    mod_bin2 = bin2[:rand_idx] + bin1[rand_idx:]
    if flag1 == True:
        mod_bin1 = 'b'+mod_bin1
    if flag2 == True:
        mod_bin2 = 'b'+mod_bin2
    return mod_bin1, mod_bin2

### Mutation of a point

In [59]:
def mutation(bin1:str) -> str:
    flag = False
    if bin1.startswith('b'):
        bin1 = bin1[1:]
        flag = True
    bin1 = list(bin1)
    rand_idx = np.random.choice(len(bin1))
    if bin1[rand_idx] == '1':
        bin1[rand_idx] = '0'
    else :
        bin1[rand_idx] = '1'
    if flag == True:
        return 'b'+"".join(bin1)
    else:
        return "".join(bin1)

### Get children after Crossover

In [64]:
def get_children_crossover(S, ncross):
    S_cross = []
    while len(S_cross) < ncross:
        idx1, idx2 = np.random.choice(len(S), 2, replace=False)
        p1, p2 = S[idx1], S[idx2]
        rand_dim = np.random.choice([0,1])
        el1, el2 = p1[rand_dim], p2[rand_dim]
        bin1, bin2 = decimal_to_binary(int(el1*100)), decimal_to_binary(int(el2*100))
        bin1, bin2 = pad_zeros(bin1, bin2)
        child1, child2 = crossover(bin1, bin2)
        dec1, dec2 = binary_to_decimal(child1)/100, binary_to_decimal(child2)/100
        if rand_dim == 0: 
            S_cross.append(np.array([dec1, p1[1]]))
            S_cross.append(np.array([dec2, p2[1]]))
        else:
            S_cross.append(np.array([p1[0], dec1]))
            S_cross.append(np.array([p2[0], dec2]))
    S_cross = np.array(S_cross)
    return S_cross

### Get children after Mutation

In [68]:
def get_child_mutation(S, nmut):
    S_mut = []
    indx_ls = np.random.choice(len(S),nmut,replace=False)
    for i in range(nmut):
        p = S[indx_ls[i]]
        rand_dim = np.random.choice([0,1])
        if rand_dim == 0:
            el = int(p[0]*100)
            bin1 = decimal_to_binary(el)
            child = mutation(bin1)
            dec1 = binary_to_decimal(child)
            p1 = np.array([dec1,p[1]])
        else:
            el = int(p[1]*100)
            bin1 = decimal_to_binary(el)
            child = mutation(bin1)
            dec1 = binary_to_decimal(child)
            p1 = np.array([p[0],dec1]) 
        S_mut.append(p1)
    S_mut = np.array(S_mut)
    return S_mut

### Main GA function

In [69]:
def genetic_algorithm(Xmin, Xmax, m=100, ncross=50, nmut=50, maxitr=10):
    S = np.round(initialise(Xmin, Xmax, m),2)
    f = np.array([obj_function(x,y) for x,y in zip(np.ravel(S[:,0]),np.ravel(S[:,1]))])
    for itr in range(maxitr):
        S_cross = get_children_crossover(S, ncross)
        S_mut = get_child_mutation(S, nmut)
        S_total = np.vstack((S,S_cross,S_mut))
        f = np.array([obj_function(x,y) for x,y in zip(np.ravel(S_total[:,0]),np.ravel(S_total[:,1]))])
        sorted_idx = np.argsort(f)
        S_total = S_total[sorted_idx]
        contour_plot(obj_function, Xmin, Xmax, S, itr)
        S = S_total[:m]
    plt.close('all')
    return S[0], f[0]


In [70]:
Xmin = [-10,-10]; Xmax = [10,10]
genetic_algorithm(Xmin, Xmax,m=200,ncross=50,nmut=50,maxitr=15)

(array([0.95, 0.  ]), 2.5800385572706084)

## Particle Swarm Optimisation (PSO)

### Initialise the population

In [87]:
def initialisation(obj_function, Xmin, Xmax, m):
    x_1 = np.random.uniform(low=Xmin[0],high=Xmax[0],size=(m,1))
    x_2 = np.random.uniform(low=Xmin[1],high=Xmax[1],size=(m,1))
    x = np.hstack((x_1,x_2))
    v = np.zeros((m,2))
    p = x.copy()
    f = np.array([obj_function(x,y) for x,y in zip(x_1,x_2)])
    best_idx = np.argmin(f)
    g = x[best_idx]
    return x, v, p, g

### Main PSO function

In [96]:
def pso(obj_function, Xmin, Xmax, m=100, omega=0.1, phi_p=0.5, phi_g=0.6, maxitr=10):
    x, v, p, g = initialisation(obj_function, Xmin, Xmax, m)
    for itr in range(maxitr):
        for i in range(m):
            beta, gamma = np.random.random(), np.random.random()
            v[i] = omega*v[i] + phi_p*beta*(p[i] - x[i]) + phi_g*gamma*(g - x[i])
            x[i] = x[i] + v[i]
            if obj_function(x[i,0],x[i,1]) < obj_function(p[i,0],p[i,1]):
                p[i] = x[i]
                if obj_function(p[i,0],p[i,1]) < obj_function(g[0],g[1]):
                    g = p[i]
        contour_plot(obj_function, Xmin, Xmax, x, itr)
    plt.close('all')
    return g, obj_function(g[0],g[1])

In [98]:
Xmin, Xmax = np.array([-10,-10]), np.array([10,10])
pso(obj_function, Xmin, Xmax, m=100, omega=0.2, phi_p=0.6, phi_g=1, maxitr=10)

(array([2.73098154e-04, 7.26586857e-05]), 0.000801435725087174)

In [124]:
## Himmbelblau's Function
def my_obj_func(x,y):
    z = (x**2+y-11)**2+(x+y**2-7)**2
    return z 

In [129]:
pso(my_obj_func, Xmin, Xmax, m=100, omega=0.1, phi_p=0.6, phi_g=0.9, maxitr=10)

(array([-3.77936069, -3.28317176]), 1.770813259655041e-07)